In [17]:
import pandas as pd
import re
from dateutil.parser import parse


In [10]:
# Charger le fichier Excel
fichier = "../data/2023_01_01_ExportRechLibre.xlsx"  # Remplace par autre chemin exact si besoin
df = pd.read_excel(fichier)

In [11]:
# Ne garder que les colonnes utiles pour ce traitement
df_actes = df[['Identifiant passage unique', 'Actes']].dropna(subset=['Actes'])


In [12]:
# Initialiser un tableau pour les événements extraits
event_log = []

In [22]:
# Regex pour extraire les couples (activité, timestamp)
# pattern = r"(.*?)(\(\d{2}/\d{2}/\d{4} \d{2}:\d{2}\))"
#Simplifier la regex pour les autres cas d'activités avec plusieurs timestamps
pattern = r"([^\(]+)\(([^)]+)\)"

In [18]:
# Boucle ligne par ligne
for _, row in df_actes.iterrows():
    case_id = row['Identifiant passage unique']
    texte = row['Actes']

    matches = re.findall(pattern, texte)
#separateur de timestamps '|'
    for activity_raw, time_strs in matches:
        activity = activity_raw.strip().lower()
        activity = re.sub(r"[^a-zA-Z0-9éèàçùêîôâ\- ]", "", activity)

        timestamps = time_strs.split('|')
        for ts in timestamps:
            ts_clean = ts.strip()

            # Test simple : on ne garde que les formats date/heure attendus, pour les cas ou on a des chiffres qui ne sont pas des timestamps
            try:
                start_time = pd.to_datetime(ts_clean, format="%d/%m/%Y %H:%M")
                event_log.append({
                    "case_id": case_id,
                    "activity": activity,
                    "start_timestamp": start_time
                })
            except ValueError:
                # Ignorer les parenthèses qui ne contiennent pas de dates
                continue

In [19]:
# Conversion en DataFrame
event_log_df = pd.DataFrame(event_log)

# Affichage d’un aperçu
event_log_df.sort_values(by=['case_id', 'start_timestamp']).head(10)

,case_id,activity,start_timestamp
22,569611027,prise de la température,2023-01-02 00:23:00
23,569611027,surveillance scope spo2 2 heures,2023-01-02 00:23:00
24,569611027,prélèvement sécrétions,2023-01-02 00:23:00
305,569611027,prise de la température,2023-01-02 00:23:00
306,569611027,surveillance scope spo2 2 heures,2023-01-02 00:23:00
307,569611027,prélèvement sécrétions,2023-01-02 00:23:00
25,569611027,résultats des tdr,2023-01-02 00:46:00
26,569611027,dispensation de médicament po,2023-01-02 00:46:00
308,569611027,résultats des tdr,2023-01-02 00:46:00
309,569611027,dispensation de médicament po,2023-01-02 00:46:00


In [20]:
from rapidfuzz import process
#Construire un dictionnaire des catégories connues
#Définir le dictionnaire expert
expert_categories = {
    "plaster": "bandage",
    "injection": "injection",
    "inhalation": "inhalation",
    "suture": "suture",
    "surgery": "surgery",
    "bandage": "bandage",
    "cleaning": "cleaning",
    "intubation": "intubation",
    "burns": "burns",
    "catherization": "catherization",
    "drug dispensation": "drug dispensation",
    "surveillance": "monitoring",
    "support": "support"
}
known_activities = list(expert_categories.keys())

def classify_activity(activity):
    try:
        if not isinstance(activity, str) or activity.strip() == "":
            return "inconnu"
        match, score, _ = process.extractOne(activity, known_activities)
        if match and score >= 85:
            return expert_categories[match]
        else:
            return "autre"
    except Exception as e:
        return f"erreur: {e}"

# Appliquer la classification
event_log_df['category'] = event_log_df['activity'].apply(classify_activity)

# Afficher un aperçu
print(event_log_df.head())

     case_id                   activity     start_timestamp category
0  569611249    prise de la température 2023-01-01 11:03:00    autre
1  569611249     mesure ponctuelle spo2 2023-01-01 11:03:00    autre
2  569611249  crp capillaire resultat45 2023-01-01 12:13:00    autre
3  569611249         pose de patch emla 2023-01-01 12:28:00    autre
4  569611259         désinfection plaie 2023-01-01 10:27:00    autre


In [21]:
from rapidfuzz import process